# Pinch Detection V2 - Robust Z-Score Fusion Algorithm

This notebook implements the advanced pinch detection algorithm based on robust statistics and multi-sensor fusion.

## Architecture Overview
- **Multi-sensor fusion**: Combine accelerometer and gyroscope signals
- **Robust statistics**: Use median/MAD instead of mean/std for noise resilience
- **Adaptive thresholding**: Dynamic threshold based on signal characteristics
- **Two-sensor gating**: Require both accel and gyro activation
- **Refractory period**: Prevent double-counting

## Key Parameters (from analysis)
- Sampling rate: ~100 Hz
- HP window: 0.5s moving mean
- Gates: acc_hp ≥ 0.04g, gyro_mag ≥ 0.15 rad/s
- Score threshold: median + 7×MAD
- Refractory: 250ms

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Pinch Detection V2 - Loaded successfully")
print("Based on robust z-score fusion architecture")

## Data Loading and Preprocessing

Load wrist motion data and extract key signals:
- userAcceleration (already gravity-compensated)
- rotationRate (gyroscope)
- Compute magnitudes and derivatives

In [ ]:
def load_wrist_data(filepath):
    """Load and preprocess wrist motion data"""
    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} samples from {filepath}")
    print(f"Columns: {list(df.columns)}")
    
    # Extract time and calculate sampling rate
    t = df['time'].values
    dt = np.median(np.diff(t))
    fs = 1.0 / dt
    print(f"Sampling rate: {fs:.1f} Hz")
    print(f"Duration: {t[-1] - t[0]:.2f} s")
    
    # Extract acceleration and gyroscope signals
    # Note: Using accelerationX/Y/Z for now, but userAcceleration is preferred
    acc_cols = ['accelerationX', 'accelerationY', 'accelerationZ']
    gyro_cols = ['rotationRateX', 'rotationRateY', 'rotationRateZ']
    
    acc_xyz = df[acc_cols].values
    gyro_xyz = df[gyro_cols].values
    
    # Compute magnitudes
    acc_mag = np.sqrt(np.sum(acc_xyz**2, axis=1))
    gyro_mag = np.sqrt(np.sum(gyro_xyz**2, axis=1))
    
    return {
        'time': t,
        'fs': fs,
        'acc_xyz': acc_xyz,
        'gyro_xyz': gyro_xyz,
        'acc_mag': acc_mag,
        'gyro_mag': gyro_mag,
        'df': df
    }

# Load the data
DATA_PATH = "~/Downloads/WristMotion.csv"
data = load_wrist_data(DATA_PATH)

## Robust Statistics Functions

Implement robust statistics for noise-resilient signal processing:
- Median Absolute Deviation (MAD)
- Robust z-scores
- Running statistics for real-time processing

In [ ]:
def robust_mad(x, axis=None, constant=1.4826):
    """Compute Median Absolute Deviation (MAD)"""
    median = np.median(x, axis=axis, keepdims=True)
    mad = np.median(np.abs(x - median), axis=axis) * constant
    return mad

def robust_zscore(x, window_size=None):
    """Compute robust z-scores using median and MAD"""
    if window_size is None:
        # Global statistics
        median = np.median(x)
        mad = robust_mad(x)
        mad = np.maximum(mad, 1e-6)  # Prevent division by zero
        return (x - median) / mad
    else:
        # Rolling statistics
        z_scores = np.zeros_like(x)
        for i in range(len(x)):
            start = max(0, i - window_size + 1)
            end = i + 1
            window = x[start:end]
            if len(window) > 1:
                median = np.median(window)
                mad = robust_mad(window)
                mad = max(mad, 1e-6)
                z_scores[i] = (x[i] - median) / mad
        return z_scores

def running_percentile(x, window_size, percentile=95):
    """Compute running percentile for adaptive thresholding"""
    result = np.zeros_like(x)
    for i in range(len(x)):
        start = max(0, i - window_size + 1)
        end = i + 1
        window = x[start:end]
        result[i] = np.percentile(window, percentile)
    return result

print("Robust statistics functions implemented")
print("- robust_mad(): Median Absolute Deviation")
print("- robust_zscore(): Robust z-scores with optional rolling window")
print("- running_percentile(): Adaptive percentile computation")

## Signal Processing Pipeline

Implement the core signal processing steps:
1. High-pass filtering to remove drift
2. Derivative computation
3. Robust z-score normalization
4. Score fusion

In [ ]:
def high_pass_filter(x, fs, window_size=0.5):
    """Remove low-frequency drift using moving average subtraction"""
    window_samples = int(window_size * fs)
    # Create moving average
    ma = np.convolve(x, np.ones(window_samples)/window_samples, mode='same')
    # Subtract to get high-pass effect
    return x - ma

def compute_derivative(x, fs):
    """Compute time derivative using finite differences"""
    dt = 1.0 / fs
    # Use central differences where possible
    dx = np.gradient(x, dt)
    return dx

def process_signals(data, hp_window=0.5, zscore_window=None):
    """Process raw signals through the full pipeline"""
    fs = data['fs']
    acc_mag = data['acc_mag']
    gyro_mag = data['gyro_mag']
    
    print(f"Processing signals with fs={fs:.1f} Hz")
    
    # Step 1: High-pass filter accelerometer to remove residual gravity/drift
    acc_hp = high_pass_filter(acc_mag, fs, hp_window)
    print(f"Applied high-pass filter (window={hp_window}s)")
    
    # Step 2: Compute derivatives (jerk-like terms)
    acc_deriv = compute_derivative(acc_hp, fs)
    gyro_deriv = compute_derivative(gyro_mag, fs)
    print("Computed derivatives")
    
    # Step 3: Compute robust z-scores
    if zscore_window is not None:
        window_samples = int(zscore_window * fs)
        z_acc = robust_zscore(acc_hp, window_samples)
        z_gyro = robust_zscore(gyro_mag, window_samples)
        z_acc_deriv = robust_zscore(acc_deriv, window_samples)
        z_gyro_deriv = robust_zscore(gyro_deriv, window_samples)
        print(f"Computed robust z-scores (rolling window={zscore_window}s)")
    else:
        z_acc = robust_zscore(acc_hp)
        z_gyro = robust_zscore(gyro_mag)
        z_acc_deriv = robust_zscore(acc_deriv)
        z_gyro_deriv = robust_zscore(gyro_deriv)
        print("Computed robust z-scores (global)")
    
    # Step 4: Clamp negative values (ignore negative-direction noise)
    z_acc_pos = np.maximum(z_acc, 0)
    z_gyro_pos = np.maximum(z_gyro, 0)
    z_acc_deriv_pos = np.maximum(z_acc_deriv, 0)
    z_gyro_deriv_pos = np.maximum(z_gyro_deriv, 0)
    
    # Step 5: Compute fusion score
    score = np.sqrt(z_acc_pos**2 + z_gyro_pos**2 + 
                   z_acc_deriv_pos**2 + z_gyro_deriv_pos**2)
    print("Computed fusion score")
    
    return {
        'acc_hp': acc_hp,
        'gyro_mag': gyro_mag,
        'acc_deriv': acc_deriv,
        'gyro_deriv': gyro_deriv,
        'z_acc': z_acc,
        'z_gyro': z_gyro,
        'z_acc_deriv': z_acc_deriv,
        'z_gyro_deriv': z_gyro_deriv,
        'score': score
    }

# Process the signals
processed = process_signals(data, hp_window=0.5, zscore_window=2.0)
print(f"\nSignal processing complete!")
print(f"Score range: {processed['score'].min():.2f} to {processed['score'].max():.2f}")

## Event Detection Algorithm

Implement the complete detection pipeline:
1. Adaptive thresholding
2. Two-sensor gating
3. Refractory period
4. Duration constraints

In [ ]:
class PinchDetector:
    def __init__(self, fs, acc_gate=0.04, gyro_gate=0.15, 
                 threshold_method='mad', threshold_k=7,
                 refractory_ms=250, min_duration_ms=20, max_duration_ms=200,
                 adaptive_window=2.0):
        """
        Pinch detection algorithm with adaptive thresholding and multi-sensor gating
        
        Parameters:
        - fs: sampling rate (Hz)
        - acc_gate: minimum acceleration threshold (g)
        - gyro_gate: minimum gyroscope threshold (rad/s)
        - threshold_method: 'mad' or 'percentile'
        - threshold_k: multiplier for MAD-based threshold
        - refractory_ms: minimum time between events (ms)
        - min/max_duration_ms: event duration constraints (ms)
        - adaptive_window: window for adaptive threshold (s)
        """
        self.fs = fs
        self.acc_gate = acc_gate
        self.gyro_gate = gyro_gate
        self.threshold_method = threshold_method
        self.threshold_k = threshold_k
        self.refractory_samples = int(refractory_ms * fs / 1000)
        self.min_duration_samples = int(min_duration_ms * fs / 1000)
        self.max_duration_samples = int(max_duration_ms * fs / 1000)
        self.adaptive_window_samples = int(adaptive_window * fs)
        
        print(f"PinchDetector initialized:")
        print(f"  Sampling rate: {fs:.1f} Hz")
        print(f"  Gates: acc≥{acc_gate:.3f}g, gyro≥{gyro_gate:.3f}rad/s")
        print(f"  Threshold: {threshold_method} (k={threshold_k})")
        print(f"  Refractory: {refractory_ms}ms ({self.refractory_samples} samples)")
        print(f"  Duration: {min_duration_ms}-{max_duration_ms}ms")
    
    def compute_adaptive_threshold(self, score):
        """Compute adaptive threshold based on signal statistics"""
        if self.threshold_method == 'percentile':
            threshold = running_percentile(score, self.adaptive_window_samples, 95)
        else:  # MAD-based
            threshold = np.zeros_like(score)
            for i in range(len(score)):
                start = max(0, i - self.adaptive_window_samples + 1)
                end = i + 1
                window = score[start:end]
                if len(window) > 10:  # Need enough samples for robust stats
                    median = np.median(window)
                    mad = robust_mad(window)
                    threshold[i] = median + self.threshold_k * mad
                else:
                    threshold[i] = np.percentile(score[:end], 90)  # Fallback
        
        return threshold
    
    def detect_events(self, processed_signals):
        """Detect pinch events using the full algorithm"""
        score = processed_signals['score']
        acc_hp = processed_signals['acc_hp']
        gyro_mag = processed_signals['gyro_mag']
        
        # Step 1: Compute adaptive threshold
        threshold = self.compute_adaptive_threshold(score)
        
        # Step 2: Find candidate peaks
        above_threshold = score > threshold
        
        # Step 3: Apply two-sensor gating
        acc_gate_met = acc_hp > self.acc_gate
        gyro_gate_met = gyro_mag > self.gyro_gate
        valid_candidates = above_threshold & acc_gate_met & gyro_gate_met
        
        # Step 4: Find event boundaries and apply duration constraints
        events = []
        last_event_end = -self.refractory_samples
        
        i = 0
        while i < len(valid_candidates):
            if valid_candidates[i] and i > (last_event_end + self.refractory_samples):
                # Find start of event
                start = i
                while start > 0 and score[start-1] > threshold[start-1] * 0.5:
                    start -= 1
                
                # Find end of event
                end = i
                while end < len(score)-1 and score[end+1] > threshold[end+1] * 0.5:
                    end += 1
                
                duration = end - start + 1
                
                # Apply duration constraints
                if self.min_duration_samples <= duration <= self.max_duration_samples:
                    # Find peak within event
                    peak_idx = start + np.argmax(score[start:end+1])
                    
                    event = {
                        'peak_idx': peak_idx,
                        'start_idx': start,
                        'end_idx': end,
                        'duration_samples': duration,
                        'peak_score': score[peak_idx],
                        'peak_acc': acc_hp[peak_idx],
                        'peak_gyro': gyro_mag[peak_idx],
                        'threshold_at_peak': threshold[peak_idx]
                    }
                    events.append(event)
                    last_event_end = end
                
                i = end + 1
            else:
                i += 1
        
        # Add timing information
        for event in events:
            event['peak_time'] = event['peak_idx'] / self.fs
            event['duration_ms'] = (event['duration_samples'] / self.fs) * 1000
        
        print(f"\nEvent detection complete:")
        print(f"  {len(events)} events detected")
        print(f"  Threshold range: {threshold.min():.2f} to {threshold.max():.2f}")
        
        return events, threshold

# Initialize detector with analysis-based parameters
detector = PinchDetector(
    fs=data['fs'],
    acc_gate=0.04,      # 0.04g from analysis
    gyro_gate=0.15,     # 0.15 rad/s from analysis
    threshold_k=7,      # median + 7*MAD from analysis
    refractory_ms=250,  # 250ms from analysis
    adaptive_window=2.0 # 2s adaptive window
)

# Detect events
events, threshold = detector.detect_events(processed)

## Visualization and Analysis

Visualize the detection results and analyze performance

In [ ]:
def plot_detection_results(data, processed, events, threshold):
    """Create comprehensive visualization of detection results"""
    t = data['time']
    
    # Convert events to arrays for plotting
    if events:
        event_times = [e['peak_time'] for e in events]
        event_scores = [e['peak_score'] for e in events]
        event_accs = [e['peak_acc'] for e in events]
        event_gyros = [e['peak_gyro'] for e in events]
    else:
        event_times = event_scores = event_accs = event_gyros = []
    
    fig, axes = plt.subplots(4, 1, figsize=(14, 12))
    
    # Plot 1: Score and threshold
    axes[0].plot(t, processed['score'], 'b-', linewidth=0.8, label='Fusion Score')
    axes[0].plot(t, threshold, 'r--', linewidth=1.0, label='Adaptive Threshold')
    if event_times:
        axes[0].scatter(event_times, event_scores, c='red', s=50, zorder=5, label=f'Events ({len(events)})')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Fusion Score with Adaptive Threshold and Detected Events')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: High-pass accelerometer
    axes[1].plot(t, processed['acc_hp'], 'g-', linewidth=0.8, label='HP Acceleration')
    axes[1].axhline(y=detector.acc_gate, color='orange', linestyle='--', alpha=0.7, label=f'Gate: {detector.acc_gate:.3f}g')
    if event_times:
        axes[1].scatter(event_times, event_accs, c='red', s=50, zorder=5)
    axes[1].set_ylabel('Acceleration (g)')
    axes[1].set_title('High-Pass Filtered Acceleration Magnitude')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Plot 3: Gyroscope
    axes[2].plot(t, processed['gyro_mag'], 'm-', linewidth=0.8, label='Gyro Magnitude')
    axes[2].axhline(y=detector.gyro_gate, color='orange', linestyle='--', alpha=0.7, label=f'Gate: {detector.gyro_gate:.3f}rad/s')
    if event_times:
        axes[2].scatter(event_times, event_gyros, c='red', s=50, zorder=5)
    axes[2].set_ylabel('Angular Rate (rad/s)')
    axes[2].set_title('Gyroscope Magnitude')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    # Plot 4: Z-scores
    axes[3].plot(t, processed['z_acc'], 'g-', alpha=0.7, linewidth=0.6, label='Z-score Acc')
    axes[3].plot(t, processed['z_gyro'], 'm-', alpha=0.7, linewidth=0.6, label='Z-score Gyro')
    axes[3].plot(t, processed['z_acc_deriv'], 'c-', alpha=0.7, linewidth=0.6, label='Z-score Acc Deriv')
    axes[3].plot(t, processed['z_gyro_deriv'], 'y-', alpha=0.7, linewidth=0.6, label='Z-score Gyro Deriv')
    axes[3].set_ylabel('Z-score')
    axes[3].set_xlabel('Time (s)')
    axes[3].set_title('Individual Z-scores')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def analyze_events(events, data):
    """Analyze detected events and print statistics"""
    if not events:
        print("No events detected!")
        return
    
    # Basic statistics
    n_events = len(events)
    duration = data['time'][-1] - data['time'][0]
    rate_per_min = (n_events / duration) * 60
    
    # Event characteristics
    durations_ms = [e['duration_ms'] for e in events]
    peak_scores = [e['peak_score'] for e in events]
    peak_accs = [e['peak_acc'] for e in events]
    peak_gyros = [e['peak_gyro'] for e in events]
    
    # Inter-event intervals
    if n_events > 1:
        event_times = [e['peak_time'] for e in events]
        ieis = np.diff(event_times)
        median_iei = np.median(ieis)
        mean_iei = np.mean(ieis)
    else:
        median_iei = mean_iei = 0
    
    print(f"\n=== EVENT ANALYSIS ===")
    print(f"Total events: {n_events}")
    print(f"Recording duration: {duration:.2f} s")
    print(f"Event rate: {rate_per_min:.1f} events/min")
    print(f"")
    print(f"Duration statistics:")
    print(f"  Median: {np.median(durations_ms):.1f} ms")
    print(f"  Range: {np.min(durations_ms):.1f} - {np.max(durations_ms):.1f} ms")
    print(f"")
    print(f"Peak acceleration:")
    print(f"  Median: {np.median(peak_accs):.3f} g")
    print(f"  Range: {np.min(peak_accs):.3f} - {np.max(peak_accs):.3f} g")
    print(f"")
    print(f"Peak gyroscope:")
    print(f"  Median: {np.median(peak_gyros):.3f} rad/s")
    print(f"  Range: {np.min(peak_gyros):.3f} - {np.max(peak_gyros):.3f} rad/s")
    
    if n_events > 1:
        print(f"")
        print(f"Inter-event intervals:")
        print(f"  Median: {median_iei:.3f} s ({60/median_iei:.1f}/min)")
        print(f"  Mean: {mean_iei:.3f} s ({60/mean_iei:.1f}/min)")
    
    # Gate compliance
    acc_compliant = sum(1 for e in events if e['peak_acc'] >= detector.acc_gate)
    gyro_compliant = sum(1 for e in events if e['peak_gyro'] >= detector.gyro_gate)
    both_compliant = sum(1 for e in events if e['peak_acc'] >= detector.acc_gate and e['peak_gyro'] >= detector.gyro_gate)
    
    print(f"")
    print(f"Gate compliance:")
    print(f"  Acceleration ≥ {detector.acc_gate:.3f}g: {acc_compliant}/{n_events} ({100*acc_compliant/n_events:.1f}%)")
    print(f"  Gyroscope ≥ {detector.gyro_gate:.3f}rad/s: {gyro_compliant}/{n_events} ({100*gyro_compliant/n_events:.1f}%)")
    print(f"  Both gates: {both_compliant}/{n_events} ({100*both_compliant/n_events:.1f}%)")

# Visualize results
plot_detection_results(data, processed, events, threshold)

# Analyze events
analyze_events(events, data)

## Parameter Tuning Interface

Interactive parameter tuning for optimization

In [ ]:
def tune_parameters(acc_gate_range=[0.02, 0.06], gyro_gate_range=[0.1, 0.25], 
                   threshold_k_range=[5, 9], refractory_range=[150, 350]):
    """Test different parameter combinations and compare results"""
    
    results = []
    
    # Test parameter grid
    acc_gates = np.linspace(acc_gate_range[0], acc_gate_range[1], 3)
    gyro_gates = np.linspace(gyro_gate_range[0], gyro_gate_range[1], 3)
    threshold_ks = np.linspace(threshold_k_range[0], threshold_k_range[1], 3)
    refractories = np.linspace(refractory_range[0], refractory_range[1], 3)
    
    print("Parameter tuning in progress...")
    
    for acc_gate in acc_gates:
        for gyro_gate in gyro_gates:
            for threshold_k in threshold_ks:
                for refractory_ms in refractories:
                    # Create detector with these parameters
                    test_detector = PinchDetector(
                        fs=data['fs'],
                        acc_gate=acc_gate,
                        gyro_gate=gyro_gate,
                        threshold_k=threshold_k,
                        refractory_ms=refractory_ms
                    )
                    
                    # Detect events
                    test_events, _ = test_detector.detect_events(processed)
                    
                    # Store results
                    results.append({
                        'acc_gate': acc_gate,
                        'gyro_gate': gyro_gate,
                        'threshold_k': threshold_k,
                        'refractory_ms': refractory_ms,
                        'n_events': len(test_events),
                        'events_per_min': (len(test_events) / (data['time'][-1] - data['time'][0])) * 60
                    })
    
    # Convert to DataFrame for analysis
    df_results = pd.DataFrame(results)
    
    print(f"\nTested {len(results)} parameter combinations")
    print(f"Event count range: {df_results['n_events'].min():.0f} - {df_results['n_events'].max():.0f}")
    print(f"Event rate range: {df_results['events_per_min'].min():.1f} - {df_results['events_per_min'].max():.1f} events/min")
    
    # Find optimal parameters (closest to expected ~60 events/min from analysis)
    target_rate = 60  # events per minute from analysis
    df_results['rate_error'] = np.abs(df_results['events_per_min'] - target_rate)
    optimal_idx = df_results['rate_error'].idxmin()
    optimal_params = df_results.iloc[optimal_idx]
    
    print(f"\n=== OPTIMAL PARAMETERS (closest to {target_rate} events/min) ===")
    print(f"Acceleration gate: {optimal_params['acc_gate']:.3f} g")
    print(f"Gyroscope gate: {optimal_params['gyro_gate']:.3f} rad/s")
    print(f"Threshold multiplier: {optimal_params['threshold_k']:.1f}")
    print(f"Refractory period: {optimal_params['refractory_ms']:.0f} ms")
    print(f"Resulting rate: {optimal_params['events_per_min']:.1f} events/min ({optimal_params['n_events']:.0f} events)")
    
    return df_results, optimal_params

# Run parameter tuning
tuning_results, optimal_params = tune_parameters()

## Export Results and Event Candidates

Export detected events for further analysis

In [ ]:
def export_events(events, data, filename="pinch_events_v2.csv"):
    """Export detected events to CSV for analysis"""
    if not events:
        print("No events to export")
        return
    
    # Create DataFrame
    event_data = []
    for i, event in enumerate(events):
        event_data.append({
            'event_id': i + 1,
            'peak_time': event['peak_time'],
            'peak_idx': event['peak_idx'],
            'duration_ms': event['duration_ms'],
            'peak_score': event['peak_score'],
            'peak_acceleration': event['peak_acc'],
            'peak_gyroscope': event['peak_gyro'],
            'threshold_at_peak': event['threshold_at_peak'],
            'start_idx': event['start_idx'],
            'end_idx': event['end_idx']
        })
    
    df_events = pd.DataFrame(event_data)
    
    # Add inter-event intervals
    df_events['inter_event_interval'] = df_events['peak_time'].diff()
    
    # Save to CSV
    df_events.to_csv(filename, index=False)
    print(f"Exported {len(events)} events to {filename}")
    
    return df_events

def create_summary_report(events, data, processed):
    """Create comprehensive summary report"""
    duration = data['time'][-1] - data['time'][0]
    n_events = len(events)
    
    report = f"""
=== PINCH DETECTION V2 SUMMARY REPORT ===

Algorithm: Robust Z-Score Fusion with Adaptive Thresholding
Data file: WristMotion.csv
Recording duration: {duration:.2f} seconds
Sampling rate: {data['fs']:.1f} Hz
Total samples: {len(data['time'])}

DETECTION PARAMETERS:
- Acceleration gate: {detector.acc_gate:.3f} g
- Gyroscope gate: {detector.gyro_gate:.3f} rad/s
- Threshold method: {detector.threshold_method} (k={detector.threshold_k})
- Refractory period: {detector.refractory_samples/data['fs']*1000:.0f} ms
- Duration constraints: {detector.min_duration_samples/data['fs']*1000:.0f}-{detector.max_duration_samples/data['fs']*1000:.0f} ms

DETECTION RESULTS:
- Total events detected: {n_events}
- Detection rate: {(n_events/duration)*60:.1f} events/minute
"""
    
    if events:
        durations_ms = [e['duration_ms'] for e in events]
        peak_accs = [e['peak_acc'] for e in events]
        peak_gyros = [e['peak_gyro'] for e in events]
        
        report += f"""
EVENT CHARACTERISTICS:
- Median duration: {np.median(durations_ms):.1f} ms
- Duration range: {np.min(durations_ms):.1f} - {np.max(durations_ms):.1f} ms
- Median peak acceleration: {np.median(peak_accs):.3f} g
- Acceleration range: {np.min(peak_accs):.3f} - {np.max(peak_accs):.3f} g
- Median peak gyroscope: {np.median(peak_gyros):.3f} rad/s
- Gyroscope range: {np.min(peak_gyros):.3f} - {np.max(peak_gyros):.3f} rad/s
"""
        
        if n_events > 1:
            event_times = [e['peak_time'] for e in events]
            ieis = np.diff(event_times)
            report += f"""
INTER-EVENT INTERVALS:
- Median IEI: {np.median(ieis):.3f} s ({60/np.median(ieis):.1f}/min)
- Mean IEI: {np.mean(ieis):.3f} s ({60/np.mean(ieis):.1f}/min)
- IEI range: {np.min(ieis):.3f} - {np.max(ieis):.3f} s
"""
    
    print(report)
    return report

# Export results
exported_events = export_events(events, data)
summary_report = create_summary_report(events, data, processed)

# Display first few events
if not exported_events.empty:
    print("\n=== FIRST 5 DETECTED EVENTS ===")
    print(exported_events.head().to_string(index=False))

## Real-time Implementation Notes

For watchOS implementation:

### Core Motion Setup
```swift
// Use CMDeviceMotion for already-processed signals
motionManager.deviceMotionUpdateInterval = 0.01 // 100 Hz
motionManager.showsDeviceMovementDisplay = true
```

### Key Algorithm Components
1. **Sliding window statistics** (2-4 second windows)
2. **Integer-optimized math** where possible
3. **Minimal memory footprint** (O(window_size) storage)
4. **Real-time robust statistics** (running median/MAD)

### Performance Targets
- **Latency**: <200ms detection-to-feedback
- **Accuracy**: >85% precision/recall
- **Battery**: Compatible with workout session duration

### Calibration Workflow
1. **Initial setup**: 30-45s of deliberate pinches
2. **Adaptive thresholds**: Based on user's typical signal levels
3. **Refractory tuning**: Based on user's typical pace
